In [ ]:
# %% [Step 1: Install Dependencies]
!pip install mp-api scikit-learn matplotlib plotly shap numpy pandas pymatgen --quiet

# %% [Step 2: Import Libraries]
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from mp_api.client import MPRester
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import shap
import getpass
from pymatgen.core import Composition

# %% [Step 3: API Connection Setup]
API_KEY = getpass.getpass("Enter your Materials Project API key:")
mpr = MPRester(API_KEY)

# %% [Step 4: Data Fetching (Validated Fields)]
try:
    docs = []
    element_combinations = [
        ["O", "C", "H", "S"], ["O", "C", "H"], ["O", "C", "S"], ["O", "H", "S"], ["C", "H", "S"],
        ["O", "C"], ["O", "H"], ["O", "S"], ["C", "H"], ["C", "S"], ["H", "S"], ["O", "C", "H", "S", "N"],
        ["O", "C", "H", "S", "P"], ["O", "C", "H", "S", "F"], ["O", "C", "H", "S", "Cl"], ["O", "C", "H", "S", "Br"],
        ["O", "C", "H", "S", "I"], ["N", "C", "H"], ["N", "O", "H"], ["N", "C", "O"], ["N", "C", "S"],
        ["N", "H", "S"], ["N", "O", "S"], ["N", "C"], ["N", "O"], ["N", "S"], ["P", "C", "H"], ["P", "O", "H"],
        ["P", "C", "O"], ["P", "C", "S"], ["P", "H", "S"], ["P", "O", "S"], ["P", "C"], ["P", "O"], ["P", "S"],
        ["F", "C", "H"], ["F", "O", "H"], ["F", "C", "O"], ["F", "C", "S"], ["F", "H", "S"], ["F", "O", "S"],
        ["F", "C"], ["F", "O"], ["F", "S"], ["Cl", "C", "H"], ["Cl", "O", "H"], ["Cl", "C", "O"], ["Cl", "C", "S"],
        ["Cl", "H", "S"], ["Cl", "O", "S"], ["Cl", "C"], ["Cl", "O"], ["Cl", "S"], ["Br", "C", "H"], ["Br", "O", "H"],
        ["Br", "C", "O"], ["Br", "C", "S"], ["Br", "H", "S"], ["Br", "O", "S"], ["Br", "C"], ["Br", "O"], ["Br", "S"],
        ["I", "C", "H"], ["I", "O", "H"], ["I", "C", "O"], ["I", "C", "S"], ["I", "H", "S"], ["I", "O", "S"],
        ["I", "C"], ["I", "O"], ["I", "S"], ["B", "C", "H"], ["B", "O", "H"], ["B", "C", "O"], ["B", "C", "S"],
        ["B", "H", "S"], ["B", "O", "S"], ["B", "C"], ["B", "O"], ["B", "S"], ["Si", "C", "H"], ["Si", "O", "H"],
        ["Si", "C", "O"], ["Si", "C", "S"], ["Si", "H", "S"], ["Si", "O", "S"], ["Si", "C"], ["Si", "O"], ["Si", "S"]
    ]
    for elements in element_combinations:
        docs.extend(mpr.materials.summary.search(
            elements=elements,
            fields=[
                "material_id", "formula_pretty", "volume",
                "band_gap", "formation_energy_per_atom",
                "symmetry", "density", "energy_above_hull",
                "theoretical"
            ],
            chunk_size=1000,
            num_chunks=20  # Increased the number of chunks to fetch more data
        ))
except Exception as e:
    print(f"API Error: {str(e)}")
    raise

print(f"Successfully fetched {len(docs)} materials")

# %% [Step 5: Data Validation & Feature Engineering]
crystal_system_mapping = {
    'Triclinic': 0, 'Monoclinic': 1, 'Orthorhombic': 2,
    'Tetragonal': 3, 'Trigonal': 4, 'Hexagonal': 5, 'Cubic': 6
}

valid_data = []
for doc in docs:
    try:
        if None not in [doc.volume, doc.band_gap, doc.formation_energy_per_atom]:
            crystal_system = doc.symmetry.crystal_system.value if doc.symmetry else 'Unknown'
            crystal_code = crystal_system_mapping.get(crystal_system, -1)

            valid_data.append({
                "formula": doc.formula_pretty,
                "volume": doc.volume,
                "band_gap": doc.band_gap,
                "density": doc.density,
                "formation_energy": doc.formation_energy_per_atom,
                "stability": doc.formation_energy_per_atom / (1 + doc.energy_above_hull),
                "symmetry_group_size": len(doc.symmetry.point_group) if doc.symmetry else 0,
                "crystal_system": crystal_code,
                "theoretical": int(doc.theoretical)
            })
    except Exception as e:
        print(f"Skipping document: {str(e)}")
        continue

df = pd.DataFrame(valid_data).astype({
    'volume': float,
    'band_gap': float,
    'density': float,
    'formation_energy': float,
    'stability': float,
    'symmetry_group_size': int,
    'crystal_system': int,
    'theoretical': int
})

# Filter invalid crystal systems
df = df[df['crystal_system'] != -1]

# %% [Step 6: Calculate Raw Adsorption Using Chemical Descriptors]
METALS = ['Pb', 'Hg', 'Cd', 'As']
CHEMICAL_DESCRIPTORS = {
    'electronegativity': {'Pb': 2.33, 'Hg': 2.00, 'Cd': 1.69, 'As': 2.18},
    'atomic_radius': {'Pb': 175, 'Hg': 151, 'Cd': 148, 'As': 115},  # in pm (picometers)
    'ionization_energy': {'Pb': 715, 'Hg': 1007, 'Cd': 867, 'As': 947}  # in kJ/mol
}

def calculate_adsorption(formula):
    comp = Composition(formula)
    elements = {e.symbol: comp.get_atomic_fraction(e) for e in comp.elements}
    electronegativity = sum(elements.get(e, 0) * CHEMICAL_DESCRIPTORS['electronegativity'].get(e, 0) for e in elements)
    atomic_radius = sum(elements.get(e, 0) * CHEMICAL_DESCRIPTORS['atomic_radius'].get(e, 0) for e in elements)
    ionization_energy = sum(elements.get(e, 0) * CHEMICAL_DESCRIPTORS['ionization_energy'].get(e, 0) for e in elements)

    adsorption_values = {}
    for metal in METALS:
        adsorption_values[metal] = (
            CHEMICAL_DESCRIPTORS['electronegativity'][metal] * electronegativity +
            CHEMICAL_DESCRIPTORS['atomic_radius'][metal] * atomic_radius +
            CHEMICAL_DESCRIPTORS['ionization_energy'][metal] * ionization_energy
        ) * 0.01  # Hypothetical scaling factor for mg/g

    total_adsorption = sum(adsorption_values.values()) * 0.25  # Hypothetical scaling factor for total adsorption

    return total_adsorption, adsorption_values

adsorption_potentials = df['formula'].apply(calculate_adsorption)
df['adsorption_potential'] = adsorption_potentials.apply(lambda x: x[0])
df['Pb_absorption'] = adsorption_potentials.apply(lambda x: x[1]['Pb'])
df['Hg_absorption'] = adsorption_potentials.apply(lambda x: x[1]['Hg'])
df['Cd_absorption'] = adsorption_potentials.apply(lambda x: x[1]['Cd'])
df['As_absorption'] = adsorption_potentials.apply(lambda x: x[1]['As'])

# Remove unrealistic negative adsorption values
df = df[df['adsorption_potential'] >= 0]

# %% [Step 7: Machine Learning Pipeline]
features = ['volume', 'band_gap', 'density', 'stability',
           'symmetry_group_size', 'crystal_system', 'theoretical']

X = df[features]
y = df[['formation_energy', 'adsorption_potential']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Multi-output regression model
model = RandomForestRegressor(n_estimators=500, max_depth=30, random_state=42)
model.fit(X_train, y_train)

# %% [Step 8: Model Evaluation]
y_pred = model.predict(X_test)

print("Model Performance:")
print(f"Formation Energy MSE: {mean_squared_error(y_test.iloc[:,0], y_pred[:,0]):.4f}")
print(f"Adsorption Potential MSE: {mean_squared_error(y_test.iloc[:,1], y_pred[:,1]):.4f}")
print(f"Combined R² Score: {r2_score(y_test, y_pred):.4f}")

# %% [Step 10: Optimal Material Selection]
df['pred_adsorption'] = model.predict(X)[:,1]
top_materials = df.drop_duplicates(subset=['formula']).nlargest(5, 'pred_adsorption')[['formula', 'pred_adsorption', 'Pb_absorption', 'Hg_absorption', 'Cd_absorption', 'As_absorption'] + features]

print("\nTop Adsorption Materials:")
print(top_materials.to_string(index=False))